In [52]:
""" import requests
from bs4 import BeautifulSoup
import pandas as pd 

# Season 2024 / All games
# The number of points of a player
url = "https://www.teamrankings.com/nba/player-stat/points"
# Fetching HTML content:
response = requests.get(url)
if response.status_code == 200:
    print("Done! - Successfully fetched the webpage.")
else:
    print(f"Failed to fetch the webpage: {response.status_code}")

# Parsing HTML content:
soup = BeautifulSoup(response.content, 'html.parser')

# Inspect page and find table containing needed data (Assuming the data is in a table.):
table = soup.find('table')
type(table)

# Extract headers:
headers = []
for th in table.find('thead').find_all('th'):
    headers.append(th.text.strip())

# Extract rows:
rows = []
for tr in table.find('tbody').find_all('tr'):
    cells = tr.find_all('td')
    row = [cell.text.strip() for cell in cells]
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows, columns=headers)

# Save data to CSV:
df.to_csv('player-stat_2024.csv', index=False)

print("Data successfully scraped and saved to 'player-stat_2024.csv'")



 """

' import requests\nfrom bs4 import BeautifulSoup\nimport pandas as pd \n\n# Season 2024 / All games\n# The number of points of a player\nurl = "https://www.teamrankings.com/nba/player-stat/points"\n# Fetching HTML content:\nresponse = requests.get(url)\nif response.status_code == 200:\n    print("Done! - Successfully fetched the webpage.")\nelse:\n    print(f"Failed to fetch the webpage: {response.status_code}")\n\n# Parsing HTML content:\nsoup = BeautifulSoup(response.content, \'html.parser\')\n\n# Inspect page and find table containing needed data (Assuming the data is in a table.):\ntable = soup.find(\'table\')\ntype(table)\n\n# Extract headers:\nheaders = []\nfor th in table.find(\'thead\').find_all(\'th\'):\n    headers.append(th.text.strip())\n\n# Extract rows:\nrows = []\nfor tr in table.find(\'tbody\').find_all(\'tr\'):\n    cells = tr.find_all(\'td\')\n    row = [cell.text.strip() for cell in cells]\n    rows.append(row)\n\n# Create DataFrame\ndf = pd.DataFrame(rows, columns=h

In [53]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

def extraction_webscrapping(url, output_file, header_tag, keep_columns=None):
    response = requests.get(url)
    if response.status_code == 200:
        print("Successfully fetched the webpage.")
    else:
        print(f"Failed to fetch the webpage: {response.status_code}")

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    
    headers = []
    for th in table.find('thead').find_all(header_tag):
        headers.append(th.text.strip())
    print(f"Headers found: {headers}")
    
    rows = []
    for tr in table.find('tbody').find_all('tr'):
        cells = tr.find_all('td')
        row = [cell.text.strip() for cell in cells]
        rows.append(row)
    
    df = pd.DataFrame(rows, columns=headers)
    
    if keep_columns:
        print(f"Filtering to keep columns: {keep_columns}")
        df = df[keep_columns]
    
    output_path = os.path.join('csv', output_file)
    #df.to_csv(,output_file, index=False)
    df.to_csv(output_path, index=False)
    print(f"Data successfully scraped and saved to {output_file}")
    print(f"Data successfully scraped")

def extract_wnba_player_salaries(url, output_file):
    response = requests.get(url)
    if response.status_code == 200:
        print("Successfully fetched the webpage.")
    else:
        print(f"Failed to fetch the webpage: {response.status_code}")

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    
    headers = ["Player", "2024 Salary"]
    rows = []
    
    for tr in table.find('tbody').find_all('tr'):
        cells = tr.find_all('td')
        name = cells[0].text.strip().split('\n')[0]  # Only take first part for the name
        salary = cells[1].text.strip().split()[0]    # Only take the salary amount
        row = [name, salary]
        rows.append(row)
    
    df = pd.DataFrame(rows, columns=headers)
    df.to_csv(output_file, index=False)
    print(f"Data successfully scraped and saved to {output_file}")

#Number of points of a player (per game)
extraction_webscrapping("https://www.teamrankings.com/nba/player-stat/points", 'player-stat_2024.csv', 'th')
#Effective field goal % (per game):
extraction_webscrapping("https://www.teamrankings.com/nba/player-stat/efg-percentage", 'efg-percentage_2024.csv', 'th')
#Assists (per game):
extraction_webscrapping("https://www.teamrankings.com/nba/player-stat/assists", 'assists_2024.csv', 'th')
#Win score (per game):
extraction_webscrapping("https://www.teamrankings.com/nba/player-stat/win-score", 'win_score_2024.csv', 'th')
#Minutes played (per game):
extraction_webscrapping("https://www.teamrankings.com/nba/player-stat/minutes-played",'minutes_played.csv','th')
# 'rebounds-offensive.csv'
extraction_webscrapping("https://www.teamrankings.com/nba/player-stat/rebounds-offensive",'rebounds-offensive.csv','th')
# 'rebounds-defensive.csv'
extraction_webscrapping("https://www.teamrankings.com/nba/player-stat/rebounds-defensive",'rebounds-defensive.csv','th')
# 'blocks.csv'
extraction_webscrapping("https://www.teamrankings.com/nba/player-stat/blocks",'blocks.csv','th')
# steals
extraction_webscrapping("https://www.teamrankings.com/nba/player-stat/steals",'steals.csv','th')

Successfully fetched the webpage.
Headers found: ['Rank', 'Player', 'Team', 'Pos', 'Value']
Data successfully scraped and saved to player-stat_2024.csv
Data successfully scraped
Successfully fetched the webpage.
Headers found: ['Rank', 'Player', 'Team', 'Pos', 'Value']
Data successfully scraped and saved to efg-percentage_2024.csv
Data successfully scraped
Successfully fetched the webpage.
Headers found: ['Rank', 'Player', 'Team', 'Pos', 'Value']
Data successfully scraped and saved to assists_2024.csv
Data successfully scraped
Successfully fetched the webpage.
Headers found: ['Rank', 'Player', 'Team', 'Pos', 'Value']
Data successfully scraped and saved to win_score_2024.csv
Data successfully scraped
Successfully fetched the webpage.
Headers found: ['Rank', 'Player', 'Team', 'Pos', 'Value']
Data successfully scraped and saved to minutes_played.csv
Data successfully scraped
Successfully fetched the webpage.
Headers found: ['Rank', 'Player', 'Team', 'Pos', 'Value']
Data successfully scrap

In [56]:
import pandas as pd

############  READ THE CSV ###########
#Assists per game
df_assists = pd.read_csv(os.path.join('csv', 'assists_2024.csv'))
df_assists.rename(columns={'Value':'AST' } , inplace=True)
display('Assists per game')
display(df_assists)

#Points per game
df_player_stats = pd.read_csv(os.path.join('csv', 'player-stat_2024.csv'))
df_player_stats.rename(columns={'Value':'PTS' } , inplace=True)
display('Points per game')
display(df_player_stats)

#Games Played  --> Minutes Played ?!
df_minutes_played = pd.read_csv(os.path.join('csv', 'minutes_played.csv'))
df_player_stats.rename(columns={'Value':'Minutes_played_per_game' } , inplace=True)
display('Minutes Played')
display(df_minutes_played)

#Offensive rebounds (per game):
df_OR = pd.read_csv(os.path.join('csv', 'rebounds-offensive.csv'))
df_OR.rename(columns={'Value':'ORB' } , inplace=True)
display('Offensive rebounds per game')
display(df_OR)
#Defensive rebounds (per game):
df_DRB = pd.read_csv(os.path.join('csv', 'rebounds-defensive.csv'))
df_DRB.rename(columns={'Value':'DRB' } , inplace=True)
display('Defensive rebounds per game')
display(df_DRB)
#Blocks (per game):
df_BLK = pd.read_csv(os.path.join('csv', 'blocks.csv'))
df_BLK.rename(columns={'Value':'BLK' } , inplace=True)
display('Blocks per game')
display(df_BLK)
#Steal (per game):
df_STL = pd.read_csv(os.path.join('csv', 'steals.csv'))
df_STL.rename(columns={'Value':'STL' } , inplace=True)
display('Steal per game')
display(df_STL)

############  Merge ###########

#Merge vs points and assist (merged):
df_points_and_assists = df_assists.merge(df_player_stats, on=["Player" , "Pos", "Team"])
display("Points and assist (merged):")
display(df_points_and_assists)

#Final result: df_points_and_assists merged with df_OR (offensive rebounds) sorted by Points_per_game:
second_merge = df_points_and_assists.merge(df_OR , on=["Player" , "Pos", "Team"])
second_merge.rename(columns={'Value':'Minutes_played_per_game' } , inplace=True)
second_merge.sort_values(by=['PTS']).reset_index()
# Remove of columns : 
second_merge.drop(columns = ['Rank_x', 'Rank_y','Rank','Team','Pos'] , axis=1, inplace=True)
# Rounds every numerical columns to 1 decimal
second_merge = second_merge.round(1)

#Finally, Offensive PER computation
second_merge['O_PER'] = round((second_merge['AST'] + second_merge['PTS']  + second_merge['ORB'])/3 ,1)
display("Points_and_assist merged with offensive rebounds and sorted by Points_per_game PTS (offensive PER rounded):")
display(second_merge)
#Final csv offense
second_merge.to_csv('nba_player_salaries_2024.csv')

##### Defense Merge #####
# a) Merge of PTS and df_DRB
defense = df_player_stats.merge(df_DRB, on=["Player" , "Team"])

# b) Merge defense and df_STL
defense = defense.merge(df_STL , on=["Player"])
defense.drop(columns = ['Rank_x', 'Team_x','Pos_x' ,'Rank_y' ,'Pos_y' ,'Rank', 'Team_y' ,'Pos'] , axis=1, inplace=True)
defense


'Assists per game'

,Rank,Player,Team,Pos,AST
0,1,Tyrese Haliburton,Indiana Pacers,PG,10.42
1,2,Luka Doncic,Dallas Mavericks,SF,9.39
2,3,Nikola Jokic,Denver Nuggets,C,8.92
3,4,James Harden,Los Angeles Clippers,SG,8.49
4,5,LeBron James,Los Angeles Lakers,SF,8.28
...,...,...,...,...,...
95,96,Karl-Anthony Towns,Minnesota Timberwolves,C,2.96
96,97,Cam Thomas,Brooklyn Nets,SG,2.91
97,98,Joe Ingles,Orlando Magic,SF,2.87
98,99,Bruce Brown,Toronto Raptors,PG,2.84


'Points per game'

,Rank,Player,Team,Pos,PTS
0,1,Luka Doncic,Dallas Mavericks,SF,32.66
1,2,Giannis Antetokounmpo,Milwaukee Bucks,SF,30.44
2,3,Shai Gilgeous-Alexander,Oklahoma City Thunder,PG,30.07
3,4,Jalen Brunson,New York Knicks,PG,29.26
4,5,Devin Booker,Phoenix Suns,SG,27.10
...,...,...,...,...,...
95,96,Cameron Johnson,Brooklyn Nets,PF,13.38
96,97,Corey Kispert,Washington Wizards,SF,13.38
97,98,Grayson Allen,Phoenix Suns,SG,13.26
98,99,Andrew Wiggins,Golden State Warriors,SF,13.21


'Minutes Played'

...

'Points and assist (merged):'

,Rank_x,Player,Team,Pos,AST,Rank_y,PTS
0,1,Tyrese Haliburton,Indiana Pacers,PG,10.42,38,19.87
1,2,Luka Doncic,Dallas Mavericks,SF,9.39,1,32.66
2,3,Nikola Jokic,Denver Nuggets,C,8.92,8,26.69
3,4,James Harden,Los Angeles Clippers,SG,8.49,62,16.91
4,5,LeBron James,Los Angeles Lakers,SF,8.28,14,25.74
...,...,...,...,...,...,...,...
66,94,Tobias Harris,Philadelphia 76ers,SF,2.99,65,16.45
67,95,Grayson Allen,Phoenix Suns,SG,2.97,98,13.26
68,96,Karl-Anthony Towns,Minnesota Timberwolves,C,2.96,31,21.21
69,97,Cam Thomas,Brooklyn Nets,SG,2.91,26,22.45


'Points_and_assist merged with offensive rebounds and sorted by Points_per_game PTS (offensive PER rounded):'

,Player,AST,PTS,ORB,O_PER
0,Nikola Jokic,8.9,26.7,2.9,12.8
1,Domantas Sabonis,8.2,19.4,3.6,10.4
2,Giannis Antetokounmpo,6.5,30.4,2.7,13.2
3,Scottie Barnes,6.0,19.8,2.4,9.4
4,Paolo Banchero,5.3,22.9,1.1,9.8
5,Zion Williamson,5.0,23.1,1.8,10.0
6,Jimmy Butler,5.0,20.7,1.8,9.2
7,Alperen Sengun,5.0,21.1,2.9,9.7
8,Pascal Siakam,4.2,21.7,1.9,9.3
9,Bam Adebayo,3.9,19.3,2.2,8.5


,Player,PTS,DRB,STL
0,Luka Doncic,32.66,8.48,1.52
1,Giannis Antetokounmpo,30.44,8.84,1.19
2,Shai Gilgeous-Alexander,30.07,4.87,1.92
3,Devin Booker,27.10,3.67,0.94
4,Kevin Durant,27.08,6.06,0.90
5,Donovan Mitchell,27.06,4.34,1.75
6,Nikola Jokic,26.69,9.57,1.37
7,De'Aaron Fox,26.64,3.70,2.04
8,Jayson Tatum,26.47,7.54,1.02
9,Anthony Edwards,26.21,4.97,1.32
